## Sorting Algorithms Application: Greedy Algorithms
-------

Optimization problems ask you to find a solution which maximizes or minimizes some objective functions. 
**Greedy algorithms** adopt a natural approach to find a optimal solution.

Mentioning Wikipedia:
*A greedy algorithm is any algorithm that follows the problem-solving heuristic of making the locally optimal choice at each stage.[1] In many problems, a greedy strategy does not usually produce an optimal solution, but nonetheless, a greedy heuristic may yield locally optimal solutions that approximate a globally optimal solution in a reasonable amount of time.*

------

### Exercise: Activity Selection Problem
Activity selection problem is a problem in which a person has a list of works to do. 

Each of the activities has a starting time and ending time. 

We need to schedule the activities in such a way the person can complete a maximum number of activities. 

Since the timing of the activities  may overlap, so it might not be possible to complete all the activities and thus we need to schedule the activities in such a way that the maximum number of activities can be finished.

Consider the following example.
![alt text](activity_table.png "Example")

A valid selection is activities 1 and 3. Hoverver, an optimal one is 2, 5, 1.

We can illustrate the problem by drawing each activity as a rectangle whose left
and right $x$-coordinates show the start and finish times. The goal is to find a
largest subset of rectangles that do not overlap vertically.

![alt text](activity.png "Example")

There is an easy **Greedy algorithm** (you'll learn more in theory lectures soon) to find an optimal solution.

- Sort activities by finishing time;
- Iterate over the activities and select any activity that does not conflict with the previously selected ones.

The algorithm runs in $\Theta(n\log n)$ time.

Your goal is to implement and test this algorithm.

Intuitively, we’d like the first activity to finish as early as possible,
because that leaves us with the most remaining activities.

If this greedy strategy works, it suggests the following very simple
algorithm. Scan through the activities in order of finish time; whenever you
encounter a activity that doesn’t conflict with your latest activity so far, take it!

The above algorithm runs in $\Theta(n\log n)$ time.

To prove that this algorithm actually gives us a maximal conflict-free selection, we use an
*exchange argument*.

We are not claiming that the greedy selection is the only maximal selection; there could be others, e.g., see figure above.

All we can claim is that at least one of the maximal selections is the one that the
greedy algorithm produces.

**Lemma** 
*At least one maximal conflict-free selection includes the activity that finishes first.*

The proof is as follows. 

- Let $f$ be the activity that finishes first. Suppose we have a maximal conflict-free selection $X$ that does not include $f$.

- Let $g$ be the first activity in $X$ to finish.

- Since $f$ finishes before $g$ does, $f$ cannot conflict with any activity in the set $X \setminus \{g\}$.

- Thus, the selection $X' = X \cup \{f\} \setminus \{g\}$ is also conflict-free.

- Since $X'$ has the same size as $X$, it is also maximal.


We use induction to complete the proof.

**Lemma** 
*The greedy selection is an optimal selection.*

The proof is as follows.

- Let $f$ be the activity that finishes first, and let $L$ be the subset of activities the start after $f$ finishes.

- The previous lemma implies that some optimal selection contains $f$, so the best selection that contains $f$ is an optimal selection.

- The best selection that includes $f$ must contain an optimal selection for the activities that do not conflict with $f$, that is, an optimal selection for $L$.

- The greedy algorithm chooses $f$ and then, by the inductive hypothesis, computes an optimal selection of activities from $L$.

**This is a general proof approach** The basic structure of this correctness proof is an inductive exchange argument which applies to several other problems. 

- Assume that there is an optimal solution that is different from the greedy solution.
- Find the ``first'' difference between the two solutions.
- Argue that we can exchange the optimal choice for the greedy choice without degrading the solution.

This argument implies by induction that some optimal solution that contains the entire greedy solution, and therefore equals the greedy solution.

-----

**You goal**: implement the above solution to find the optimal selection from a list of activities, where each activity is a pair.

In [2]:
def maxActivities(A):
    act = []
    A.sort(key=lambda e: e[1]) #ordino per finish time
    for elem in A:
        if len(act)==0 or elem[0]>= act[-1][1]: #se la lista è vuota o se lo starting time è dopo il finish time dell'ultima attività
            act.append(elem)
    return act

In [3]:
A = [[4,6], [0,2], [1,3], [1,6], [3,4]]
print(maxActivities(A))

[[0, 2], [3, 4], [4, 6]]


In [4]:
assert maxActivities(A) == [[0, 2], [3, 4], [4, 6]], "Fail!"

---

### Exercise: Fractional Knapsack Problem

![alt text](knapsack.png "Example")

*We are given $n$ items. Each item $i$ has a value $v_i$ and a weight $w_i$. We need put a subset of these items in a knapsack of capacity $W$ to get the maximum total value in the knapsack.*

This is a very popular [problem](https://en.wikipedia.org/wiki/Knapsack_problem).

In the 0-1 Knapsack problem, we are not allowed to break items. We either take the whole item or do not take it.

In Fractional Knapsack, we can break items for maximizing the total value of knapsack.


As an example, consider three items: $v = \{ 60, 100, 120\}$ and $w = \{10, 20, 30\}$ and a knapsack of capacity $W = 50$.

The maximum possible value is $240$ obtained by taking full items of $10$ and $20$ and $2/3$rd of last item of $30$.

An efficient solution to find an optimal selection is to use the greedy approach.

The basic idea of greedy approach is to calculate the ratio value/weight for each item and sort items in decreasing order of this ratio. Then, we take the item with highest ratio and add them until we cannot add the next item as whole and at the end add the next item as much as we can.

This strategy always obtains an optimal solution of this problem.

To see why associate a rectangle to each item. The rectangle of item $i$ has a
base of size $w_i$ and a height of size $v_i$. The diagonal of this rectangle
is a segment of slope $v_i/w_i$.

Consider now any selection of items whose total weight equals $W$.

We can sort the selected items in order of their ratio and draw the diagonals of their rectangles, one after the other.

There cannot exist any assignment whose drawn is above the one of the greedy selection.

![alt text](knapsack_proof.png "Example of proof")

Instead, 0-1 Knapsack problem is NP-Hard.

**Your goal:**
Write a function ```fractional_knapsack(L,W)``` which takes a list L of pairs *(value, weight)* and the capacity $W$ and returns maximum possible value we can obtain by selecting items.  

In [9]:
def fractional_knapsack(L, capacity):
    index = list(range(len(L)))
    ratio = list()
    for el in L:
        ratio.append(el[0]/el[1])
    index.sort(key=lambda i: ratio[i], reverse=True) #ordino in maniera decrescente gli indici dei tuple in base al ratio
    
    cumul_value = 0
    for i in index:
        if L[i][1] <= capacity: #se il peso dell'oggetto corrente è minore della capacity totale
            cumul_value += L[i][0] #valore cumulato degli oggetti
            capacity -= L[i][1] #diminuisco la capacity totale di una quantità pari al peso dell'oggetto inserito nello zaino
        else:
            cumul_value += L[i][0]*(capacity/L[i][1]) #se supero la capacity totale, sommo solo una frazione del valore dell'oggetto in base alla frazione di capacity rimanente
            break
    return cumul_value

In [10]:
L = [(60, 10), (100, 20), (120, 30)] #(valore, peso)
print(fractional_knapsack(L, 50))

240.0


In [6]:
## Test your implementation here

L = [(60, 10), (100, 20), (120, 30)]

assert fractional_knapsack(L, 50) == 240.0, "Fail!"

L = [(30, 5), (40, 10), (45, 15), (77, 22), (90, 25)]

assert fractional_knapsack(L, 60) == 230.0, "Fail!"

assert fractional_knapsack(L, 15) == 70.0,  "Fail!"

assert fractional_knapsack(L, 10) == 50.0,  "Fail!"